# 誤差逆伝播法に対応したNNを実装する

In [5]:
import sys, os
import numpy as np
from collections import OrderedDict

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.py、layers.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')

from common.layers import SoftmaxWithLoss, Affine, ReLU

ローカル環境での実行です


### [演習]
* 以下の誤差逆伝播法に対応したNNクラスを完成させましょう。

In [6]:
# ヒント

# OrderedDictを用いると、順序を保持したdictを生成することができる
# ただし、Pythonバージョン3.6以降は、普通のdictでも順序が保存される
dic1 = OrderedDict()
dic1["a"] = 1
dic1["c"] = 3
dic1["b"] = 2
for key, value in dic1.items():
    print(key, value)
print()

# dictから逆順でvalueを取り出す方法
layers = list(dic1.values())
layers.reverse() # 順序を逆転させる
print(layers)

a 1
c 3
b 2

[2, 3, 1]


In [10]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict() # 順番付きdict形式. 
        # self.layers['Affine1'] = Affine(    ,    )
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
        self.layers['Relu1'] = ReLU()
        # self.layers['Affine2'] = Affine(    ,    )
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        self.lastLayer = SoftmaxWithLoss() # 出力層
        
    def predict(self, x):
        """
        予測関数
        x : 入力
        """
        for layer in self.layers.values():
            # 入力されたxを更新していく = 順伝播計算
            # x = layer.forward(      )
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        """
        損失関数
        x:入力データ, t:正解データ
        """
        # y = self.predict(    )
        y = self.predict(x)
        # return self.lastLayer.forward(    ,    )
        return self.lastLayer.forward(y,t)
    
    def accuracy(self, x, t):
        """
        正解率
        """
        # 予測. 返り値は正規化されていない実数
        # y = self.predict(     )
        y = self.predict(x)
        
        #正規化されていない実数をもとに、最大値になるindexに変換する
        # y = np.argmax(     , axis=  )
        # データ方向ではなく、one-hotベクトルのうち、最大の要素のINDEXを1とする
        y = np.argmax(y, axis=1)
        
        # tが 0/1 ではなく連続数の場合は下記により 0/1 に変換する
        if t.ndim != 1 : 
            """
            one-hotベクトルの場合、正解データをindexに変換する
            """
            t = np.argmax(t, axis=1)
        
        # 正解率
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def gradient(self, x, t):
        """
        全パラメータの勾配を計算
        """
        
        # 順伝播
        self.loss(x, t)

        # 逆伝播
        dout = self.lastLayer.backward() # 出力層
        
        ## doutを逆向きに伝える 
        layers = list(self.layers.values())
        layers.reverse()#中身を反転。これにより、Affine2→ReLU→Affine1の順番で、下記において計算される
        for layer in layers:
            dout = layer.backward(dout)

        # dW, dbをgradsにまとめる
        grads = {}
        # grads['W1'], grads['b1'] = 
        # grads['W2'], grads['b2'] = 
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [9]:
# 学習用データ
x = np.array([[1, 2, 3],
            [3, 2, 1]])
t = np.array([[0, 0, 1],
            [1, 0, 0]])

tnet = TwoLayerNet( input_size=3, hidden_size=5, output_size=3)

print("predict=\n", tnet.predict(x))
print()
print("loss=\n", tnet.loss(x, t))
print()
print("accuracy=", tnet.accuracy(x, t))
print()
print("gradient=\n",)
grads = tnet.gradient(x, t)
for key, value in grads.items():
    print(key)
    print(value)

predict=
 [[-1.71263784e-04 -2.94125223e-05  6.65588607e-04]
 [-3.71268545e-04 -3.30834436e-04  7.06831702e-04]]

loss=
 1.0985431978422477

accuracy= 0.5

gradient=

W1
[[ 0.          0.         -0.00088597 -0.00811426  0.00601437]
 [ 0.          0.         -0.00053376 -0.01246281 -0.0012345 ]
 [ 0.          0.         -0.00018156 -0.01681137 -0.00848336]]
b1
[ 0.          0.         -0.00026688 -0.00623141 -0.00061725]
W2
[[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.00166882  0.00949407 -0.00782524]
 [-0.00514296  0.00453633  0.00060663]
 [-0.01217193  0.01974981 -0.00757788]]
b2
[-0.16678319  0.33324718 -0.16646399]
